### Connecting Azure Data Lake

In [0]:
spark.conf.set("fs.azure.account.auth.type.datatricksexternal190.dfs.core.windows.net", "OAuth")
spark.conf.set("fs.azure.account.oauth.provider.type.datatricksexternal190.dfs.core.windows.net", "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider")
spark.conf.set("fs.azure.account.oauth2.client.id.datatricksexternal190.dfs.core.windows.net", dbutils.secrets.get(scope='ali-scope',key='appsecret'))
spark.conf.set("fs.azure.account.oauth2.client.secret.datatricksexternal190.dfs.core.windows.net",dbutils.secrets.get(scope='ali-scope',key='sec-id'))
spark.conf.set("fs.azure.account.oauth2.client.endpoint.datatricksexternal190.dfs.core.windows.net", "https://login.microsoftonline.com/ce69d97e-12e0-4b6d-8f1a-0c5d7534dae8/oauth2/token")


### Load Task (GOLD LAYER)

**Accesing Silver Layer Data**

In [0]:
df_sales=spark.read.format('parquet')\
              .option("header", "true")\
              .option("inferSchema", "true")\
              .load("abfss://retailproject@datatricksexternal190.dfs.core.windows.net/Silver/retail/sales_silver") 


df_features=spark.read.format('parquet')\
              .option("header", "true")\
              .option("inferSchema", "true")\
              .load("abfss://retailproject@datatricksexternal190.dfs.core.windows.net/Silver/retail/features_silver")   


df_stores=spark.read.format('parquet')\
              .option("header", "true")\
              .option("inferSchema", "true")\
              .load("abfss://retailproject@datatricksexternal190.dfs.core.windows.net/Silver/retail/stores_silver")                           

In [0]:
df_weather=spark.read.format('parquet')\
              .option("header", "true")\
              .option("inferSchema", "true")\
              .load("abfss://retailproject@datatricksexternal190.dfs.core.windows.net/Silver/weather_silver") 

df_holidays=spark.read.format('parquet')\
              .option("header", "true")\
              .option("inferSchema", "true")\
              .load("abfss://retailproject@datatricksexternal190.dfs.core.windows.net/Silver/holidays_silver")               

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [0]:
df_sales.printSchema()
df_stores.printSchema()
df_features.printSchema()
df_weather.printSchema()
df_holidays.printSchema()

root
 |-- store_id: integer (nullable = true)
 |-- dept_id: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- weekly_sales: double (nullable = true)
 |-- IsHoliday: boolean (nullable = true)
 |-- Sales_Id: integer (nullable = true)

root
 |-- store_id: integer (nullable = true)
 |-- store_type: string (nullable = true)
 |-- store_size: integer (nullable = true)

root
 |-- store_id: integer (nullable = true)
 |-- date: date (nullable = true)
 |-- temperature: float (nullable = true)
 |-- fuel_price: float (nullable = true)
 |-- cpi: float (nullable = true)
 |-- unemployment_rate: float (nullable = true)

root
 |-- time: timestamp (nullable = true)
 |-- temperature_2m: float (nullable = true)
 |-- precipitation: float (nullable = true)

root
 |-- date: date (nullable = true)
 |-- localName: string (nullable = true)
 |-- name: string (nullable = true)
 |-- holiday_type: string (nullable = true)



In [0]:
dim_store=df_stores.select("store_id","store_type","store_size")
dim_store.printSchema()

root
 |-- store_id: integer (nullable = true)
 |-- store_type: string (nullable = true)
 |-- store_size: integer (nullable = true)



In [0]:
dim_features=df_features.select("date","fuel_price","cpi","unemployment_rate")
dim_features.printSchema()

root
 |-- date: date (nullable = true)
 |-- fuel_price: float (nullable = true)
 |-- cpi: float (nullable = true)
 |-- unemployment_rate: float (nullable = true)



In [0]:
dim_weather=df_weather.select(
        to_date(col("time"),"yyyy-MM-dd").alias("date"),
        date_format(col("time"),"HH:MM:SS").alias("time"),
        col("temperature_2m").alias("temperature"),
        col("precipitation")

)
dim_weather.printSchema()
 

root
 |-- date: date (nullable = true)
 |-- time: string (nullable = true)
 |-- temperature: float (nullable = true)
 |-- precipitation: float (nullable = true)



In [0]:
dim_holidays=df_holidays.select("date","holiday_type")
dim_holidays.printSchema()

root
 |-- date: date (nullable = true)
 |-- holiday_type: string (nullable = true)



In [0]:
sales_fact=df_sales.select("sales_id","store_id","dept_id","weekly_sales","date")
sales_fact.printSchema()

root
 |-- sales_id: integer (nullable = true)
 |-- store_id: integer (nullable = true)
 |-- dept_id: integer (nullable = true)
 |-- weekly_sales: double (nullable = true)
 |-- date: date (nullable = true)



### Storing Star Schema In Delta Format In Gold Layer

In [0]:
sales_fact.write.format('delta')\
              .mode('append')\
              .option("path", "abfss://retailproject@datatricksexternal190.dfs.core.windows.net/Gold/sales_fact")\
              .save() 


dim_store.write.format('delta')\
              .mode('append')\
              .option("path", "abfss://retailproject@datatricksexternal190.dfs.core.windows.net/Gold/dim_store")\
              .save() 



dim_features.write.format('delta')\
              .mode('append')\
              .option("path", "abfss://retailproject@datatricksexternal190.dfs.core.windows.net/Gold/dim_features")\
              .save() 
                      

In [0]:
dim_weather.write.format('delta')\
              .mode('append')\
              .option("path", "abfss://retailproject@datatricksexternal190.dfs.core.windows.net/Gold/dim_weather")\
              .save() 



dim_holidays.write.format('delta')\
              .mode('append')\
              .option("path", "abfss://retailproject@datatricksexternal190.dfs.core.windows.net/Gold/dim_holidays")\
              .save()               

In [0]:
dim_date=df_sales.select("date")\
                 .withColumn("date",to_date(col("date"),"yyyy-MM-dd"))\
                 .withColumn("year",year(col("date")))\
                 .withColumn("month",month(col("date")))\
                 .withColumn("day",dayofmonth(col("date")))\
                 .withColumn("dayofweek",date_format(col("date"),"E"))\
                 .withColumn("dayofweeknum",date_format(col("date"),"u"))\
                 .withColumn("weekofyear",weekofyear(col("date")))\
                 .withColumn("quarter",quarter(col("date")))

In [0]:

dim_date.write.format('delta')\
              .mode('append')\
              .option("path", "abfss://retailproject@datatricksexternal190.dfs.core.windows.net/Gold/dim_holidays")\
              .save() 

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-8259093923798987>, line 4
      1 dim_date.write.format('delta')\
      2               .mode('append')\
      3               .option("path", "abfss://retailproject@datatricksexternal190.dfs.core.windows.net/Gold/dim_holidays")\
----> 4               .save()

File /databricks/spark/python/pyspark/sql/connect/readwriter.py:670, in DataFrameWriter.save(self, path, format, mode, partitionBy, **options)
    668     self.format(format)
    669 self._write.path = path
--> 670 self._spark.client.execute_command(
    671     self._write.command(self._spark.client), self._write.observations
    672 )

File /databricks/spark/python/pyspark/sql/connect/client/core.py:1205, in SparkConnectClient.execute_command(self, command, observations, extra_request_metadata)
   1203     req.user_context.user_id = self._user_id
   1204 req.plan.c